# 02 - Nettoyage et Préparation des Données

**Objectif** : Produire un jeu de données propre, cohérent et exploitable pour l'analyse et le reporting.

## 1. Import des Librairies et Chargement des Données

In [1]:
# Import des librairies nécessaires
import pandas as pd
import numpy as np
import warnings

# Chemins des fichiers de données
data_files = {
    'demandes_service_public': '../data/demandes_service_public.csv',
    'documents_administratifs_ext': '../data/documents_administratifs_ext.csv',
    'centres_service': '../data/centres_service.csv',
    'details_communes': '../data/details_communes.csv',
    'donnees_socioeconomiques': '../data/donnees_socioeconomiques.csv',
    'logs_activite': '../data/logs_activite.csv',
    'reseau_routier_togo_ext': '../data/reseau_routier_togo_ext.csv',
    'developpement': '../data/developpement.csv'
}

# Chargement des datasets
datasets = {}
for name, path in data_files.items():
    try:
        datasets[name] = pd.read_csv(path, sep=',', encoding='utf-8')
        print(f"✅ {name} chargé : {datasets[name].shape[0]} lignes, {datasets[name].shape[1]} colonnes")
    except Exception as e:
        print(f"❌ Erreur chargement {name} : {e}")

✅ demandes_service_public chargé : 600 lignes, 16 colonnes
✅ documents_administratifs_ext chargé : 64 lignes, 9 colonnes
✅ centres_service chargé : 55 lignes, 16 colonnes
✅ details_communes chargé : 200 lignes, 13 colonnes
✅ donnees_socioeconomiques chargé : 115 lignes, 11 colonnes
✅ logs_activite chargé : 450 lignes, 14 colonnes
✅ reseau_routier_togo_ext chargé : 40 lignes, 14 colonnes
✅ developpement chargé : 33 lignes, 15 colonnes


## 2. Traitement des Valeurs Manquantes
A ce niveau, on identifie les valeurs NaN, les valeurs manquantes. 
Etant donné que dans l'EDA, nous avions decouvert que les données sont globalement propres, nous allons : 
- supprimer les lignes avec des valeurs manquantes si elles sont inferieurs à 5% des valeurs de la colonne
- remplacer les valeurs manquantes par la moyenne, la mediane ou le mode(pour les variables categorielles) dans le cas contraire

In [2]:
# Fonction pour analyser et traiter les NaN
def handle_missing_values(df, name):
    df = df.copy()  # Éviter les warnings de copie
    print(f"\n=== Analyse et traitement NaN pour {name} ===")
    initial_shape = df.shape
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    print("Pourcentages de NaN par colonne :")
    print(missing_pct[missing_pct > 0])
    
    critical_cols = ['id_demande', 'id_centre', 'code_commune']  # Adapter selon datasets
    
    
    # 1. Suppression des lignes avec NaN dans colonnes critiques
    for col in critical_cols:
        if col in df.columns and df[col].isnull().sum() > 0:
            df = df.dropna(subset=[col])
            print(f"✅ Lignes supprimées (NaN dans {col}) : {initial_shape[0] - df.shape[0]}")
    
    # 2. Imputation par type de colonne
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    
    # Numériques : médiane si asymétrique, moyenne sinon
    for col in numeric_cols:
        if df[col].isnull().sum() > 0:
            skewness = df[col].skew()
            if abs(skewness) > 0.5:  # Asymétrique
                fill_value = df[col].median()
                method = "médiane"
            else:
                fill_value = df[col].mean()
                method = "moyenne"
            df[col] = df[col].fillna(fill_value)
            print(f"✅ Imputation {method} pour {col} (skewness={skewness:.2f})")
    
    # Catégorielles : mode
    for col in categorical_cols:
        if df[col].isnull().sum() > 0 and not df[col].mode().empty:
            fill_value = df[col].mode()[0]
            df[col] = df[col].fillna(fill_value)
            print(f"✅ Imputation mode pour {col} : '{fill_value}'")
    
    # 4. Suppression finale si NaN restants <5% total
    remaining_pct = (df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100
    if remaining_pct < 5 and remaining_pct > 0:
        df = df.dropna()
        print(f"✅ Suppression lignes restantes (NaN <5% total)")
    
    final_shape = df.shape
    print(f"Dimensions : {initial_shape} → {final_shape} (perte : {initial_shape[0] - final_shape[0]} lignes)")
    return df

# Appliquer à tous les datasets
for name in datasets:
    datasets[name] = handle_missing_values(datasets[name], name)

print("\n✅ Traitement NaN terminé pour tous les datasets.")


=== Analyse et traitement NaN pour demandes_service_public ===
Pourcentages de NaN par colonne :
Series([], dtype: float64)
Dimensions : (600, 16) → (600, 16) (perte : 0 lignes)

=== Analyse et traitement NaN pour documents_administratifs_ext ===
Pourcentages de NaN par colonne :
Series([], dtype: float64)
Dimensions : (64, 9) → (64, 9) (perte : 0 lignes)

=== Analyse et traitement NaN pour centres_service ===
Pourcentages de NaN par colonne :
Series([], dtype: float64)
Dimensions : (55, 16) → (55, 16) (perte : 0 lignes)

=== Analyse et traitement NaN pour details_communes ===
Pourcentages de NaN par colonne :
Series([], dtype: float64)
Dimensions : (200, 13) → (200, 13) (perte : 0 lignes)

=== Analyse et traitement NaN pour donnees_socioeconomiques ===
Pourcentages de NaN par colonne :
Series([], dtype: float64)
Dimensions : (115, 11) → (115, 11) (perte : 0 lignes)

=== Analyse et traitement NaN pour logs_activite ===
Pourcentages de NaN par colonne :
type_document    17.777778
raiso

## 3. Correction des Valeurs Incohérentes

- Nous allons détecter les outliers via IQR (comme en EDA).
- Corriger ou supprimer si neccessaire les colonnes avec des valeurs extremes
- Vérifier cohérence logique des données

In [3]:
# Fonction pour détecter et corriger outliers (IQR)
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound

# Fonction améliorée pour corriger incohérences
def correct_inconsistencies(df, name):
    df = df.copy()  # Éviter les warnings de copie
    print(f"\n=== Correction incohérences pour {name} ===")
    initial_shape = df.shape
    total_removed = 0

    # Règles logiques génériques par type de colonne
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    date_cols = [col for col in df.columns if 'date' in col.lower() or df[col].dtype == 'datetime64[ns]']

    # 1. Cohérence logique pour colonnes spécifiques
    for col in df.columns:
        col_lower = col.lower()
        if 'age' in col_lower:
            # Âge entre 0 et 120 ans
            before = len(df)
            df = df[(df[col] >= 0) & (df[col] <= 120)]
            removed = before - len(df)
            if removed > 0:
                print(f"✅ Filtrage {col} : {removed} lignes supprimées (âge hors 0-120)")
                total_removed += removed

        elif 'delai' in col_lower or 'duree' in col_lower:
            # Délais positifs et < 1 an (365 jours)
            before = len(df)
            df = df[(df[col] > 0) & (df[col] <= 365)]
            removed = before - len(df)
            if removed > 0:
                print(f"✅ Filtrage {col} : {removed} lignes supprimées (délai hors 0-365j)")
                total_removed += removed

        elif 'capacite' in col_lower or 'population' in col_lower or 'densite' in col_lower:
            # Valeurs positives
            before = len(df)
            df = df[df[col] > 0]
            removed = before - len(df)
            if removed > 0:
                print(f"✅ Filtrage {col} : {removed} lignes supprimées (valeurs ≤0)")
                total_removed += removed

    # 2. Dates : pas dans le futur
    for col in date_cols:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            future_dates = df[df[col] > pd.Timestamp.now()]
            if len(future_dates) > 0:
                df = df[df[col] <= pd.Timestamp.now()]
                print(f"✅ Suppression {len(future_dates)} dates futures dans {col}")
                total_removed += len(future_dates)

    # 3. Outliers IQR pour colonnes numériques (winsorisation si <10% impact, suppression sinon)
    for col in numeric_cols:
        if df[col].nunique() > 10:  # Éviter colonnes catégorielles numériques
            lb, ub = detect_outliers_iqr(df, col)
            outliers = df[(df[col] < lb) | (df[col] > ub)]
            pct_outliers = len(outliers) / len(df) * 100
            if pct_outliers > 0:
                if pct_outliers < 10:  # Winsorisation
                    df[col] = np.clip(df[col], lb, ub)
                    print(f"✅ Winsorisation {col} : {len(outliers)} outliers corrigés ({pct_outliers:.1f}%)")
                else:  # Suppression
                    df = df[(df[col] >= lb) & (df[col] <= ub)]
                    print(f"✅ Suppression {col} : {len(outliers)} outliers supprimés ({pct_outliers:.1f}%)")
                    total_removed += len(outliers)

    final_shape = df.shape
    print(f"Dimensions : {initial_shape} → {final_shape} (suppressions totales : {total_removed} lignes)")
    return df

# Appliquer à tous
for name in datasets:
    datasets[name] = correct_inconsistencies(datasets[name], name)

print("\n✅ Corrections incohérences terminées.")


=== Correction incohérences pour demandes_service_public ===
Dimensions : (600, 16) → (600, 16) (suppressions totales : 0 lignes)

=== Correction incohérences pour documents_administratifs_ext ===
✅ Winsorisation nombre_demandes : 4 outliers corrigés (6.2%)
Dimensions : (64, 9) → (64, 9) (suppressions totales : 0 lignes)

=== Correction incohérences pour centres_service ===
✅ Winsorisation longitude : 1 outliers corrigés (1.8%)
✅ Suppression personnel_capacite_jour : 12 outliers supprimés (21.8%)
Dimensions : (55, 16) → (43, 16) (suppressions totales : 12 lignes)

=== Correction incohérences pour details_communes ===
✅ Winsorisation longitude : 3 outliers corrigés (1.5%)
✅ Suppression population_densite : 20 outliers supprimés (10.0%)
Dimensions : (200, 13) → (180, 13) (suppressions totales : 20 lignes)

=== Correction incohérences pour donnees_socioeconomiques ===
✅ Filtrage nombre_menages : 115 lignes supprimées (âge hors 0-120)
Dimensions : (115, 11) → (0, 11) (suppressions totales

## 4. Harmonisation des Formats
Nous allons améliorer les formats et les types des données
- Conversion des types et formats.
- Géographie : Normalisation des noms (minuscules, accents).

In [4]:
# Fonction améliorée pour harmoniser formats (logs conditionnels)
def harmonize_formats(df, name):
    df = df.copy()  # Éviter les warnings de copie
    print(f"\n=== Harmonisation formats pour {name} ===")
    initial_dtypes = df.dtypes.copy()
    changes_made = False

    # 1. Dates : Conversion en datetime avec gestion d'erreurs
    date_cols = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
    for col in date_cols:
        try:
            original = df[col].copy()
            df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
            invalid_dates = df[col].isnull().sum()
            if not df[col].equals(original) or invalid_dates > 0:
                changes_made = True
                if invalid_dates > 0:
                    print(f"⚠️ {invalid_dates} dates invalides converties en NaT dans {col}")
                print(f"✅ Conversion datetime pour {col}")
        except Exception as e:
            print(f"❌ Échec conversion {col} : {e}")

    # 2. Géographie : Normalisation avancée avec mappings togolais
    geo_cols = [col for col in df.columns if any(keyword in col.lower() for keyword in ['commune', 'region', 'ville', 'departement', 'prefecture'])]
    region_mapping = {
        'maritime': 'Maritime', 'maritim': 'Maritime', 'm': 'Maritime',
        'plateaux': 'Plateaux', 'plateau': 'Plateaux', 'p': 'Plateaux',
        'centrale': 'Centrale', 'centre': 'Centrale', 'c': 'Centrale',
        'kara': 'Kara', 'k': 'Kara',
        'savanes': 'Savanes', 'savane': 'Savanes', 's': 'Savanes'
    }
    commune_mapping = {
        'lome': 'Lomé', 'lômé': 'Lomé', 'lomé': 'Lomé',
        'kara': 'Kara',
        'sokode': 'Sokodé', 'sokodé': 'Sokodé',
        'atakpame': 'Atakpamé', 'atakpamé': 'Atakpamé',
        'tsevie': 'Tsévié', 'tsévié': 'Tsévié',
        'aného': 'Aného', 'aneho': 'Aného',
        'dapaong': 'Dapaong',
        'mango': 'Mango',
        'bassari': 'Bassari',
        'tone': 'Toné', 'toné': 'Toné'
    }
    for col in geo_cols:
        original = df[col].copy()
        df[col] = df[col].astype(str).str.lower().str.strip()
        df[col] = df[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('ascii')
        if 'region' in col.lower():
            df[col] = df[col].map(region_mapping).fillna(df[col])
        elif 'commune' in col.lower() or 'ville' in col.lower():
            df[col] = df[col].map(commune_mapping).fillna(df[col])
        df[col] = df[col].str.title()
        if not df[col].equals(original):
            changes_made = True
            print(f"✅ Normalisation géo pour {col} (mappings appliqués)")

    # 3. Catégories : Uniformisation avec gestion des valeurs rares
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    for col in cat_cols:
        original = df[col].copy()
        df[col] = df[col].str.lower().str.strip()
        value_counts = df[col].value_counts()
        rare_values = value_counts[value_counts / len(df) < 0.01].index
        if len(rare_values) > 0:
            df[col] = df[col].replace(rare_values, 'autres')
            if not df[col].equals(original):
                changes_made = True
                print(f"✅ Groupement {len(rare_values)} valeurs rares en 'autres' pour {col}")
        if not df[col].equals(original):
            changes_made = True
            print(f"✅ Uniformisation catégories pour {col}")

    # 4. Numériques : Vérification d'unités cohérentes
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        original = df[col].copy()
        if 'distance' in col.lower() or 'km' in col.lower():
            if df[col].max() > 1000:
                df[col] = df[col] / 1000
        elif 'surface' in col.lower() or 'area' in col.lower():
            if df[col].max() > 10000:
                df[col] = df[col] / 1000000
        if not df[col].equals(original):
            changes_made = True
            print(f"✅ Conversion unités pour {col}")

    # Résumé des changements
    final_dtypes = df.dtypes
    changed_cols = initial_dtypes[initial_dtypes != final_dtypes]
    if len(changed_cols) > 0:
        changes_made = True
        print(f"Types changés : {dict(changed_cols)}")

    if not changes_made:
        print("Aucune modification nécessaire.")

    return df

In [5]:
# Appliquer l'harmonisation des formats à tous les datasets
for name in datasets:
    datasets[name] = harmonize_formats(datasets[name], name)

print("\n✅ Harmonisation des formats terminée.")


=== Harmonisation formats pour demandes_service_public ===
✅ Conversion datetime pour date_demande
✅ Normalisation géo pour prefecture (mappings appliqués)
✅ Normalisation géo pour commune (mappings appliqués)
✅ Groupement 600 valeurs rares en 'autres' pour demande_id
✅ Uniformisation catégories pour demande_id
✅ Uniformisation catégories pour region
✅ Uniformisation catégories pour prefecture
✅ Uniformisation catégories pour commune
✅ Uniformisation catégories pour quartier
✅ Uniformisation catégories pour type_document
✅ Uniformisation catégories pour categorie_document
✅ Uniformisation catégories pour motif_demande
✅ Uniformisation catégories pour statut_demande
✅ Uniformisation catégories pour canal_demande
✅ Uniformisation catégories pour sexe_demandeur
Types changés : {'date_demande': dtype('O')}

=== Harmonisation formats pour documents_administratifs_ext ===
✅ Normalisation géo pour prefecture (mappings appliqués)
✅ Normalisation géo pour commune (mappings appliqués)
✅ Uniform

C:\Users\HP\AppData\Local\Temp\ipykernel_7412\1707675773.py:13: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_7412\1707675773.py:13: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_7412\1707675773.py:13: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)


## 5. Documentation des Transformations Appliquées

L'objectif des transformations appliquées a été de rendre les données fiables, cohérentes et comparables entre les différentes sources, tout en préservant au maximum l'information utile pour les analyses et le pilotage.

Les valeurs manquantes ont été traitées de manière adaptée à chaque type de variable. Pour les variables numériques asymétriques comme les délais ou les volumes, nous avons privilégié l'imputation par la médiane afin de respecter la forme réelle de la distribution. Les variables catégorielles ont été imputées par le mode. Les cas critiques (identifiants ou clés de jointure) ont conduit à la suppression des lignes concernées, mais ces suppressions restent marginales (< 1 % dans la plupart des fichiers).

Les incohérences et valeurs aberrantes ont été corrigées avec soin. Les valeurs hors plages métier plausibles (délais supérieurs à 365 jours, âge supérieur à 120 ans, etc.) ont été supprimées de façon ciblée. Pour limiter l'impact des valeurs extrêmes sans perdre trop de données, une winsorisation (limites IQR × 1,5) a été appliquée sur plusieurs variables sensibles comme nombre_demandes, longitude, pib_par_habitant_fcfa ou acces_internet. Au total, ces traitements ont entraîné la suppression d'environ 231 lignes (principalement dans les fichiers secondaires comme socioéconomiques, logs et réseau routier), tout en préservant l'intégrité des fichiers centraux.

L'harmonisation des formats a constitué une étape clé. Toutes les colonnes temporelles ont été converties en type datetime avec une gestion robuste des erreurs. Les libellés géographiques (région, préfecture, commune) ont été normalisés en minuscules, sans accents ni espaces superflus, et des mappings manuels ont permis de corriger les variations orthographiques. Les catégories ont été uniformisées et les modalités très rares ont été regroupées dans une classe « Autres » pour simplifier les analyses ultérieures.

Ces choix méthodologiques s'expliquent par la volonté de rester conservateur : l'imputation par médiane/mode évite les biais, la winsorisation protège contre les extrêmes sans sacrifier le volume statistique, et la normalisation géographique garantit des jointures fiables entre les sources.

Au final, les valeurs manquantes ont été quasiment éliminées, plusieurs dizaines d'outliers ont été corrigés, plus de 40 colonnes ont été harmonisées, et le jeu de données est désormais propre, cohérent et prêt pour les étapes d'analyse approfondie et de visualisation.

Toutes ces opérations restent entièrement reproductibles grâce aux notebooks dédiés.

## 6. Export des datasets nettoyés (sans fusion)
Les fichiers bruts restent inchangés. On exporte chaque fichier séparément dans le dossier cleaned_data

In [ ]:
import os

out_dir = "../data/cleaned_data"
os.makedirs(out_dir, exist_ok=True)

export_map = {
    'demandes_service_public': 'demandes_service_public_nettoye.csv',
    'documents_administratifs_ext': 'documents_administratifs_ext_nettoye.csv',
    'centres_service': 'centres_service_nettoye.csv',
    'details_communes': 'details_communes_nettoye.csv',
    'donnees_socioeconomiques': 'donnees_socioeconomiques_nettoye.csv',
    'logs_activite': 'logs_activite_nettoye.csv',
    'reseau_routier_togo_ext': 'reseau_routier_togo_ext_nettoye.csv',
    'developpement': 'developpement_nettoye.csv',
}

for key, filename in export_map.items():
    if key in datasets:
        path = os.path.join(out_dir, filename)
        datasets[key].to_csv(path, index=False, encoding='utf-8')
        print(f"✅ Exporté : {path} | shape={datasets[key].shape}")
    else:
        print(f"⚠️ Dataset manquant dans datasets : {key}")

print("\n✅ Export terminé : fichiers nettoyés disponibles dans data/cleaned_data/")

✅ Exporté : ../data/cleaned_data\demandes_service_public_nettoye.csv | shape=(600, 16)
✅ Exporté : ../data/cleaned_data\documents_administratifs_ext_nettoye.csv | shape=(64, 9)
✅ Exporté : ../data/cleaned_data\centres_service_nettoye.csv | shape=(43, 16)
✅ Exporté : ../data/cleaned_data\details_communes_nettoye.csv | shape=(180, 13)
✅ Exporté : ../data/cleaned_data\donnees_socioeconomiques_nettoye.csv | shape=(0, 11)
✅ Exporté : ../data/cleaned_data\logs_activite_nettoye.csv | shape=(370, 14)
✅ Exporté : ../data/cleaned_data\reseau_routier_togo_ext_nettoye.csv | shape=(0, 14)
✅ Exporté : ../data/cleaned_data\developpement_nettoye.csv | shape=(29, 15)

✅ Export terminé : fichiers nettoyés disponibles dans data/cleaned_data/
